In [2]:
# 日本語化ライブラリの導入

!pip install japanize-matplotlib | tail -n 1

# 共通事前処理

# 余分なワーニングを非表示にする
import warnings
warnings.filterwarnings('ignore')

# ライブりのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#matplotlibの日本語化対応
import japanize_matplotlib

#データフレーム表示用関数
from IPython.display import display

# 表示オプション調整
# NumPyの不動招集点の表示精度
np.set_printoptions(suppress=True, precision=4)
# pandasでの浮動小数点の表示精度
pd.options.display.float_format = '{:.4f}'.format
#データフレームで全ての項目を表示
pd.set_option("display.max_columns", None)
#グラフのデフォルトフォントの指定
plt.rcParams["font.size"] = 14
#乱数の種
random_seed = 123

# チューニング

　モデルの精度を上げるためのタスクが**チューニング**

分類の処理パターンに対象を絞って解説する

このチューニングが、データサイエンスの重要な業務であるが今回は基本的なチューニング方法を対象とする

### アルゴリズムの選択

　チューニング　１　：　**適切なアルゴリズムの選択**  \\
　
  経験を積めば、どのようなアルゴリズムが良いかは自然とわかってくる \\
  最初のステップは、複数のアルゴリズム候補を横並びにしてモデルを作り、精度を比較する \\
  例　：　「乳がん疾患データ」　　　　訓練データ　90%,  テストデータ 10% で作成

In [3]:
# サンプルデータの読み込み
# (乳がん疾患データ)

# データのロード
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# 入力データ : x (30次元)
# 正解データ　: y 
x = cancer.data
y = cancer.target

In [4]:
# サンプルデータの分割

# データ分割のパラメータ
test_size = 0.1

# データ分割
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, 
                                                    test_size = test_size, random_state=random_seed, stratify=y)

# 分割サイズ確認
print(x.shape)
print(x_train.shape)
print(x_test.shape)

(569, 30)
(512, 30)
(57, 30)


In [5]:
# 複数アルゴリズムで精度を比較
# 結果が同じになるよう radom_stateは同一にする

# 線形回帰
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# サポートベクターマシン（カーネル)
from sklearn.svm import SVC
algorithm2 = SVC(kernel = 'rbf', random_state=random_seed)

#　決定木
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=random_seed)

#XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)

#アルゴリズムのリスト作成
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [6]:
#　複数のアルゴリズムで精度比較
for algorithm in algorithms:

  #訓練データの学習
  algorithm.fit(x_train, y_train)

  #検証データで精度測定
  score = algorithm.score(x_test, y_test)

  # アルゴリズム名の取得
  name = algorithm.__class__.__name__

  #精度とアルゴリズム名表示
  print(f'score : {score:.4f} {name}')

score : 0.9474 LogisticRegression
score : 0.8947 SVC
score : 0.9474 DecisionTreeClassifier
score : 0.9298 RandomForestClassifier
score : 0.9825 XGBClassifier


検証結果

　・サポートベクタマシンは精度が悪い \\
　・XGBoostは精度がよく検証データに対して98.25[%]となっている \\
  ・他の3つのアルゴリズムは、似たり寄ったりだがロジスティック回帰と決定木の精度がランダムフォレストよりは良い

## ハイパーパラメータの最適化

 **ハイパーパラメター** : 各アルゴリズムが持っているパラメータの最適化

 例：　先ほど一番悪かった SVMで調整を行う

In [7]:
# デフォルトパラメータの確認
algorithm = SVC(kernel='rbf', random_state=random_seed)
print(algorithm)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)


ここで、　大きく影響を及ぼすパラメータであるgammaとCを調整する

In [8]:
# gammaの最適化
gammas = [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]

for gamma in gammas:
  algorithm = SVC(kernel='rbf', gamma = gamma, 
                  random_state= random_seed)
  algorithm.fit(x_train, y_train)
  score = algorithm.score(x_test, y_test)
  print(f'score : {score:.4f} gamma:{gamma}')


score : 0.6316 gamma:1
score : 0.6316 gamma:0.1
score : 0.6316 gamma:0.01
score : 0.9474 gamma:0.001
score : 0.9474 gamma:0.0001
score : 0.9474 gamma:1e-05


考察 

0.001より1/10倍しても変わらないので0.001を採用

In [9]:
# Cの最適化
# gammaは先ほど調べた最適値0.001を採用

Cs = [1, 10, 100, 1000, 10000]

for C in Cs:
  algorithm = SVC(kernel='rbf', gamma = 0.001,
                  C=C, random_state=random_seed)
  algorithm.fit(x_train, y_train)
  score = algorithm.score(x_test, y_test)
  print(f'score : {score:.4f} C:{C}')

score : 0.9474 C:1
score : 0.9298 C:10
score : 0.9298 C:100
score : 0.9298 C:1000
score : 0.9298 C:10000


考察

　C = 1が最適であることがわかる

以上のように**何を**モデルのチューニング対象とするかは、各アルゴリズムで違うので調べること

## 交差検定法

　アルゴリズムの比較では、XGBoostが一番良かったが、実はこれだけでは判断できない

　モデルの精度は、検証データに大きく依存している

　ある検証データに対してたまたま良い精度を出したアルゴリズム出会っても、乱数の種の値を変えて別の検証データに変更するとまりよくない可能性もある

　　このような問題に対応するための検定方法が**交差検定法(cross validatio)**

 学習データ全体に対して最初に分割数(cv)を決定して、学習データを均等に分割し、それぞれ1つのグループのみをテストデータとして検証を行う


In [10]:
# 特定のアルゴリズムに対して交差検定法を実施

# アルゴリズムの定義
algorithm = SVC(kernel='rbf', random_state=random_seed, gamma=0.001, C=1)

# 分割時に正解データの分布が偏らないようにStratifiedKFoldを利用
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

# 交差検定の実施 (分割数=3)
from sklearn.model_selection import cross_val_score
scores = cross_val_score(algorithm, x_train, y_train, cv=stratifiedkfold)

# 平均値の計算
mean = scores.mean()

# 結果の表示
print(f'平均スコア : {mean:.4f} 個別スコア : {scores}')

平均スコア : 0.9141 個別スコア : [0.8889 0.9181 0.9353]


In [11]:
# 候補アルゴリズムのリスト作成

# 複数アルゴリズムで精度を比較
# 結果が同じになるよう radom_stateは同一にする

# 線形回帰
from sklearn.linear_model import LogisticRegression
algorithm1 = LogisticRegression(random_state=random_seed)

# サポートベクターマシン（カーネル)
from sklearn.svm import SVC
algorithm2 = SVC(kernel = 'rbf', random_state=random_seed, gamma = 0.001, C = 1)

#　決定木
from sklearn.tree import DecisionTreeClassifier
algorithm3 = DecisionTreeClassifier(random_state=random_seed)

# ランダムフォレスト
from sklearn.ensemble import RandomForestClassifier
algorithm4 = RandomForestClassifier(random_state=random_seed)

#XGBoost
from xgboost import XGBClassifier
algorithm5 = XGBClassifier(random_state=random_seed)

#アルゴリズムのリスト作成
algorithms = [algorithm1, algorithm2, algorithm3, algorithm4, algorithm5]

In [12]:
# 複数アルゴリズムで精度を比較

# 分割時に正解データの分布が偏らないようにStratifieldKFoldを利用
from sklearn.model_selection import StratifiedKFold
stratifiedkfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import cross_val_score
for algorithm in algorithms:
  #　交差検定法の実施
  scores = cross_val_score(algorithm, x_train, y_train, cv=stratifiedkfold)
  score = scores.mean()
  name = algorithm.__class__.__name__
  print(f'平均スコア： {score:.4f} 個別スコア : {scores} {name}')

平均スコア： 0.9453 個別スコア : [0.9357 0.9474 0.9529] LogisticRegression
平均スコア： 0.9141 個別スコア : [0.8889 0.9181 0.9353] SVC
平均スコア： 0.9062 個別スコア : [0.8713 0.9415 0.9059] DecisionTreeClassifier
平均スコア： 0.9629 個別スコア : [0.9649 0.9591 0.9647] RandomForestClassifier
平均スコア： 0.9590 個別スコア : [0.9591 0.9649 0.9529] XGBClassifier


### グリッドサーチ

前回のチューニング : gammaとCを２回に分けて最適値を調べた

特定のパラメータ値の組み合わせが最適である可能性がある。厳密のために調べる方法

In [16]:
# グリッドリサーチに交差検定を組み合わせて最適なパラメータを探索

params = {
    'C' : [1, 10, 100, 1000, 10000],
    'gamma' : [1, 0.1, 0.01, 0.001, 0.0001, 0.00001]
}

algorithm = SVC(random_state=random_seed)

from sklearn.model_selection import StratifiedKFold
stratifieldfold = StratifiedKFold(n_splits=3)

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(algorithm, params, cv=stratifiedkfold)
gs.fit(x_train, y_train)

# ベストのモデルを取得し検証データを分類
best = gs.best_estimator_
best_pred = best.predict(x_test)
print(best)

SVC(C=1000, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma=1e-05, kernel='rbf',
    max_iter=-1, probability=False, random_state=123, shrinking=True, tol=0.001,
    verbose=False)


In [18]:
# 精度の取得
score = best.score(x_test, y_test)
print(f'スコア　: {score:.4f}')

# 混同行列を出力
from sklearn.metrics import confusion_matrix
print()
print('混同行列')
print(confusion_matrix(y_test, best_pred))

スコア　: 0.9825

混同行列
[[20  1]
 [ 0 36]]
